In [2]:
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np
from hilbert import decode, encode
import pandas as pd
from tqdm import tqdm
import torch
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key="uM0HPEvEu6OyX3dTEuB4Fihgz",
    project_name="hilbertcnn",
    workspace="romantic-brutalist",
)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/home/caliban/hilbertCNN' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.com https://www.comet.com/romantic-brutalist/hilbertcnn/97ba96d310274c4e87901f2352800402



In [3]:
import os
os.environ["COMET_API_KEY"] = "uM0HPEvEu6OyX3dTEuB4Fihgz"

In [20]:
hyper_params = {"batch_size": 64, "num_epochs": 5, "learning_rate": 0.0015,"trainset":"_4layer_w_volume","loss_metric":"MAE","regularizer":"L1"}
experiment.log_parameters(hyper_params)

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [22]:
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
class HilbertImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_labels = pd.read_parquet(os.path.join(self.img_dir, annotations_file))
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, -1])
        image = np.load(img_path)
        label = self.img_labels.iloc[idx, :-1].values.astype("float64")
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [23]:
train_ds = HilbertImageDataset(annotations_file="target_line_3.parquet",img_dir="imgs/train"+hyper_params["trainset"]+"/")
test_ds = HilbertImageDataset(annotations_file="target_line_3.parquet",img_dir="imgs/test"+hyper_params["trainset"]+"/")

In [24]:
trainloader = DataLoader(train_ds, batch_size=hyper_params["batch_size"], shuffle=True)
testloader = DataLoader(test_ds, batch_size=hyper_params["batch_size"], shuffle=True)

In [103]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(5, 8, 5,padding="same")
        self.pool = nn.AvgPool2d(3, 2)
        self.conv2 = nn.Conv2d(8, 16, 3,padding="same")
        self.conv3 = nn.Conv2d(16, 32, 3)
        self.conv4 = nn.Conv2d(32, 64, 2)
        self.conv5 = nn.Conv2d(64, 128, 2)
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 16)
        self.fc6 = nn.Linear(16, 8)
        
        self.batch1=nn.BatchNorm2d(8)
        self.batch2=nn.BatchNorm2d(16)
        self.batch3=nn.BatchNorm2d(32)
        self.batch4=nn.BatchNorm2d(64)
        
        self.dropout1=nn.Dropout2d(p=0.2)
        

    def forward(self, x):
        x = self.batch1(F.relu(self.conv1(self.dropout1(x))))
        x = self.pool(self.batch2(F.relu(self.conv2(x))))
        x = self.pool(self.batch3(F.relu(self.conv3(self.dropout1(x)))))
        x = self.batch4(F.relu(self.conv4(x)))
        x = F.relu(self.conv5(x))
        
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x


net = Net().to(device)

In [104]:
import torch.optim as optim

criterion = torch.nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=hyper_params["learning_rate"])

In [105]:
def train(model,optimizer,criterion,dataloader,epoch):
    model.train()
    running_loss = 0.0
    total_loss= 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.float().to(device)
        labels = labels.float().to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        

        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        total_loss += loss.item()

        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.8f}')
            experiment.log_metric("batch_loss", running_loss/200)
            running_loss = 0.0
    experiment.log_metrics({"loss": total_loss/len(trainloader)}, epoch=epoch)

In [106]:
with experiment.train():
    print("Running Model Training")
    for epoch in range(hyper_params["num_epochs"]):
        train(net, optimizer, criterion, trainloader, epoch)

Running Model Training
[1,   200] loss: 0.00043445
[1,   400] loss: 0.00002432
[1,   600] loss: 0.00002489
[1,   800] loss: 0.00002307
[1,  1000] loss: 0.00001989
[1,  1200] loss: 0.00001607
[1,  1400] loss: 0.00001760
[1,  1600] loss: 0.00001747
[1,  1800] loss: 0.00001470
[1,  2000] loss: 0.00001522
[1,  2200] loss: 0.00001526
[2,   200] loss: 0.00001431
[2,   400] loss: 0.00001468
[2,   600] loss: 0.00001345
[2,   800] loss: 0.00001413
[2,  1000] loss: 0.00001688
[2,  1200] loss: 0.00001391
[2,  1400] loss: 0.00001468
[2,  1600] loss: 0.00001857
[2,  1800] loss: 0.00001400
[2,  2000] loss: 0.00001435
[2,  2200] loss: 0.00001674
[3,   200] loss: 0.00001319
[3,   400] loss: 0.00001423
[3,   600] loss: 0.00001526
[3,   800] loss: 0.00001660
[3,  1000] loss: 0.00001396
[3,  1200] loss: 0.00001326
[3,  1400] loss: 0.00001515
[3,  1600] loss: 0.00001825
[3,  1800] loss: 0.00001428
[3,  2000] loss: 0.00001506
[3,  2200] loss: 0.00001457
[4,   200] loss: 0.00001395
[4,   400] loss: 0.000013

In [14]:
from comet_ml.integration.pytorch import log_model
#log_model(experiment, net, model_name="4_layer_w_volume_classic_CNN_mae_slope")

In [98]:
model_checkpoint = {
    "epoch": epoch,
    "model_state_dict": net.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "criterion":criterion}
log_model(experiment, model_checkpoint, model_name="4_layer_w_volume_classic_CNN_mse_line_3")


In [16]:
from comet_ml.integration.pytorch import load_model
model = Net()

# Load the model state dict from a Comet Experiment
checkpoint = load_model("experiment://ebb9475c2e544221bada3876eb4d95ba/4_layer_w_volume_classic_CNN_mae_line_3")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

model.train()

Net(
  (conv1): Conv2d(5, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (apool): AdaptiveMaxPool2d(output_size=4)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=8, bias=True)
  (batch1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout2d(p=0.2, inplace=False)
)

In [17]:
model.to(device)

Net(
  (conv1): Conv2d(5, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (apool): AdaptiveMaxPool2d(output_size=4)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=8, bias=True)
  (batch1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout2d(p=0.2, inplace=False)
)

In [107]:
dataiter = iter(testloader)
pd.set_option('display.max_rows', 500)
cols = list(pd.concat([pd.read_parquet("imgs/train"+hyper_params["trainset"]+"/target_line_3.parquet"),
pd.read_parquet("imgs/test"+hyper_params["trainset"]+"/target_line_3.parquet")]).iloc[:,:-1].columns)
fcs = [[i+"_target",i+"_pred"] for i in cols]
fullcols = [item for sublist in fcs for item in sublist]
tl = []
for images, labels in dataiter:
    images = images.float().to(device)
    labels = labels.float().to(device)
    outputs = net(images)
    evaldf = pd.DataFrame(labels.cpu(),columns=[i+"_target" for i in cols]).join(pd.DataFrame(outputs.cpu().tolist(),columns=[i+"_pred" for i in cols]))
    tl.append(evaldf[fullcols])
tf = pd.concat(tl)

In [108]:
tf.describe()

,Low_5_target,Low_5_pred,High_5_target,High_5_pred,Low_10_target,Low_10_pred,High_10_target,High_10_pred,Low_15_target,Low_15_pred,High_15_target,High_15_pred,Low_20_target,Low_20_pred,High_20_target,High_20_pred
count,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000,15830.000000
mean,-0.001327,-0.000737,0.001359,0.002236,-0.001321,-0.001430,0.001369,0.002536,-0.001323,-0.001121,0.001363,0.003019,-0.001317,-0.001663,0.001372,0.002528
std,0.001651,0.001046,0.001668,0.000777,0.002677,0.000749,0.002729,0.000691,0.003339,0.000756,0.003366,0.000678,0.003926,0.000828,0.003964,0.000492
min,-0.061664,-0.007691,-0.000042,0.001150,-0.057036,-0.010623,-0.025510,0.001310,-0.066908,-0.006971,-0.043833,0.002211,-0.059140,-0.007297,-0.039862,0.001875
25%,-0.001718,-0.000859,0.000385,0.001830,-0.002187,-0.001574,0.000000,0.002178,-0.002550,-0.001288,-0.000299,0.002667,-0.002799,-0.001727,-0.000541,0.002264
50%,-0.000879,-0.000486,0.000906,0.002078,-0.000905,-0.001302,0.000943,0.002396,-0.000934,-0.000980,0.000960,0.002860,-0.000927,-0.001448,0.000955,0.002423
75%,-0.000363,-0.000184,0.001752,0.002354,0.000057,-0.001038,0.002256,0.002648,0.000360,-0.000696,0.002591,0.003106,0.000636,-0.001247,0.002863,0.002609
max,0.000047,0.000428,0.043472,0.008915,0.025916,0.000087,0.059302,0.009782,0.036149,0.000171,0.049969,0.009263,0.036223,-0.000841,0.060177,0.007294


In [16]:
tf["High_Slope_pred_n"] = (tf["High_Slope_pred"]-tf["High_Slope_pred"].mean())/tf["High_Slope_pred"].std()
tf["Low_Slope_pred_n"] = (tf["Low_Slope_pred"]-tf["Low_Slope_pred"].mean())/tf["Low_Slope_pred"].std()

In [19]:
tf

,Low_5_target,Low_5_pred,High_5_target,High_5_pred,Low_10_target,Low_10_pred,High_10_target,High_10_pred,Low_15_target,Low_15_pred,High_15_target,High_15_pred,Low_20_target,Low_20_pred,High_20_target,High_20_pred
0,-0.000534,-0.000758,0.000814,0.001154,-0.000925,-0.001427,0.000603,0.000377,-0.000492,-0.001450,0.001578,0.001396,-0.000041,-0.000390,0.001482,0.002283
1,-0.000701,-0.000852,0.000240,0.001268,-0.000138,-0.001539,0.001010,0.000497,-0.000046,-0.001550,0.001107,0.001504,0.000185,-0.000488,0.001389,0.002401
2,-0.000831,-0.000877,0.000326,0.001298,-0.001573,-0.001569,0.000453,0.000529,-0.000255,-0.001576,0.002564,0.001532,0.000713,-0.000513,0.002833,0.002432
3,-0.003348,-0.000407,0.000237,0.000758,-0.002580,-0.001033,-0.000470,-0.000051,-0.000752,-0.001078,0.001318,0.000993,0.000278,-0.000019,0.001631,0.001865
4,-0.000004,-0.000877,0.002267,0.001298,0.000625,-0.001569,0.002336,0.000529,0.000599,-0.001576,0.001797,0.001532,-0.000133,-0.000513,0.001963,0.002432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,-0.000369,-0.001089,0.000773,0.001559,-0.000005,-0.001826,0.000758,0.000803,-0.000050,-0.001800,0.001874,0.001775,-0.000738,-0.000732,0.002298,0.002702
18,-0.008028,-0.001183,0.000567,0.001675,-0.005420,-0.001940,0.005737,0.000926,0.001644,-0.001900,0.007611,0.001883,0.001540,-0.000830,0.005157,0.002821
19,-0.005083,-0.001155,0.000072,0.001641,-0.007304,-0.001907,-0.003599,0.000889,-0.009294,-0.001870,-0.005343,0.001851,-0.009048,-0.000801,-0.007111,0.002786
20,-0.000278,-0.000507,0.001038,0.000863,0.000134,-0.001139,0.001349,0.000065,-0.000468,-0.001184,0.001122,0.001108,-0.000236,-0.000127,0.000598,0.001978


In [40]:
pl=[]
margin=5
for k in range(1,31):
    profit = 1
    for i,row in tf.iloc[-50*(k+1):-50*k].iterrows():
        rnd = np.random.rand()
        if abs(row.High_30_pred)>abs(row.Low_30_pred):
            if row.High_30_pred<=row.High_30_target:
                if rnd>0.3:
                    profit*=1+row.High_30_pred*margin
                else:
                    profit*=(1-0.0025*margin)
        else:
            if row.Low_30_pred>=row.Low_30_target:
                if rnd>0.3:
                    profit*=1+abs(row.Low_30_pred)*margin
                else:
                    profit*=(1-0.0025*margin)

        profit*=0.9998
    pl.append(profit)



In [41]:
pl

[1.0673509123909688,
 1.1238975005714207,
 1.0179766104314514,
 1.1487265083205547,
 1.272956367423118,
 1.1580981473119594,
 1.0877998905626058,
 1.1984989752168203,
 1.0348282767472443,
 1.047606543957888,
 1.0763505198841317,
 0.9908169899480509,
 1.0430807761777416,
 1.0931483750336155,
 1.2296696447347297,
 1.0701848705406514,
 1.1747573886805607,
 1.2059888936603302,
 1.0837346684629348,
 1.1692219806322768,
 1.1686494699488508,
 1.1047820862622317,
 1.2680866808671198,
 1.1824812169368362,
 1.0516656427751156,
 1.0866221544899486,
 1.0687716026579372,
 1.0256492916697946,
 1.2947348324054557,
 1.083971900212829]

In [42]:
np.array(pl).prod()

28.578952832384328

In [23]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/romantic-brutalist/hilbertcnn/96e1ccf1adcf4fcdb834346f05753b2f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_batch_loss [110] : (0.06814029417932034, 0.25200512740761044)
COMET INFO:     train_loss [10]        : (0.07008739145824767, 0.14456800313195575)
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 64
COMET INFO:     learning_rate : 0.0015
COMET INFO:     loss_metric   : MAE
COMET INFO:     num_epochs    : 10
COMET INFO:     trainset      : _4layer_w_volume
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     model-element       : 2 (2.09 MB)
COMET INFO:     notebook            : 1
COMET INFO:     os packages   